In [ ]:
# DAY01
f = open('input/01.txt', 'r')
lines = [line.strip() for line in f.readlines()]

MAPPING = {
    'one': 1,
    'two': 2,
    'three': 3,
    'four': 4,
    'five': 5,
    'six': 6,
    'seven': 7,
    'eight': 8,
    'nine': 9,
}

def get_digit(s, dir=1, count_words=False):
    p = 0 if dir == 1 else len(s) - 1
    while p >= 0 and p < len(s):
        if s[p].isdigit():
            return s[p]
        if count_words:
            for k, v in MAPPING.items():
                if s[p:].startswith(k):
                    return str(v)
        p += dir

def get_number(line, count_words=False):
    n = int(get_digit(line, 1, count_words)) * 10 + int(get_digit(line, -1, count_words))
    return n

res1 = sum(get_number(l) for l in lines)
print(res1)

res2 = sum(get_number(l, True) for l in lines)
print(res2)

In [ ]:
# DAY02
f = open('input/02.txt', 'r')
lines = [line.strip() for line in f.readlines()]


def parse_turn(line):
    parts = line.split(',')
    res = [0, 0, 0]
    for p in parts:
        p = p.strip()
        n, color = p.split(' ')
        color = color.strip()
        if color == 'red':
            res[0] += int(n)
        if color == 'green':
            res[1] += int(n)
        if color == 'blue':
            res[2] += int(n)
    return res


def parse_line(line):
    line = line.split(': ')[1]
    return [parse_turn(t) for t in line.split(';')]


def is_game_possible(game, target):
    for r, g, b in game:
        if r > target[0] or g > target[1] or b > target[2]:
            return False
    return True


def get_min_target(game):
    min_r = max([r for r, _, _ in game])
    min_g = max([g for _, g, _ in game])
    min_b = max([b for _, _, b in game])
    return [min_r, min_g, min_b]


games = [parse_line(l) for l in lines]

TARGET = [12, 13, 14]

res1 = 0
for i in range(len(games)):
    if is_game_possible(games[i], TARGET):
        res1 += i + 1

print(res1)

min_targets = [get_min_target(g) for g in games]
res2 = sum(max(1, r) * max(1, g) * max(1, b) for r, g, b in min_targets)
print(res2)


In [ ]:
# DAY03
f = open('input/03.txt', 'r')
lines = [line.strip() for line in f.readlines()]

def is_symbol(c):
    return c is not None and not c.isdigit() and c != '.'


OFFSETS = [
    [-1, -1], [0, -1], [1, -1],
    [-1, 0], [1, 0],
    [-1, 1], [0, 1], [1, 1]]


def get_at(lines, x, y):
    h = len(lines)
    w = len(lines[0])
    if x >= 0 and x < w and y >= 0 and y < h:
        return lines[y][x]
    return None

def has_symbol_neighbor(lines, i, j):
    h = len(lines)
    w = len(lines[0])
    for dx, dy in OFFSETS:
        x = i + dx
        y = j + dy
        if is_symbol(get_at(lines, x, y)):
            return True
    return False

def get_gear_neighbors(lines, i, j):
    res = set()
    h = len(lines)
    w = len(lines[0])
    for dx, dy in OFFSETS:
        x = i + dx
        y = j + dy
        c = get_at(lines, x, y)
        if c == '*':
            res.add((x, y))
    return res

def extract_nums(lines):
    h = len(lines)
    w = len(lines[0])
    
    is_part = False
    n = 0
    num_digits = 0
    nums = []
    gears = set()
    for j in range(h):
        for i in range(w):
            c = lines[j][i]
            
            if c.isdigit():
                n = n * 10 + int(c)
                num_digits += 1
                is_part = is_part or has_symbol_neighbor(lines, i, j)
                g = get_gear_neighbors(lines, i, j)
                if g is not None:
                    gears = gears.union(g)
                
            if (not c.isdigit() or i == w - 1) and num_digits > 0:
                nums.append((n, is_part, gears))
                is_part = False
                n = 0
                num_digits = 0
                gears = set()
    return nums


def extract_gears(nums):
    gears = {}
    for _, _, gs in nums:
        for g in gs:
            if g in gears:
                gears[g] += 1
            else:
                gears[g] = 1
                
    gears = {g: 1 for g, count in gears.items() if count == 2}
    
    for n, is_part, gs in nums:
        for g in gs:
            if g in gears:
                gears[g] *= n
    return gears
    

nums = extract_nums(lines)
res1 = sum(n for n, is_part, _ in nums if is_part)
print(res1)

gears = extract_gears(nums)
res2 = sum(gears.values())
print(res2)

In [ ]:
# DAY04
f = open('input/04.txt', 'r')
lines = [line.strip() for line in f.readlines()]

def parse_card(line):
    line = line.split(": ")[1]
    n1, n2 = line.split("|")
    player_nums = [int(x.strip()) for x in n2.strip().split()]
    winning_nums = [int(x.strip()) for x in n1.strip().split()]
    return (winning_nums, player_nums)

def get_num_matches(card):
    common = set(card[0]).intersection(set(card[1]))
    return len(common)

def get_score(card):
    num_matches = get_num_matches(card)
    return int(2 ** num_matches / 2)

def eval_num_copies(cards):
    ncards = len(cards)
    copies = [1] * ncards
    for i in range(ncards):
        nm = get_num_matches(cards[i])
        for j in range(i + 1, min(i + 1 + nm, ncards)):
            copies[j] += copies[i]
    return copies
        
cards = [parse_card(line) for line in lines]

res1 = sum(get_score(c) for c in cards)
print(res1)

copies = eval_num_copies(cards)
res2 = sum(copies)
print(res2)
        
    

In [ ]:
# DAY05
text = open('input/05.txt', 'r').read()

def parse_ints(line):
    return [int(x) for x in line.split()]
    
def parse_mapping(text):
    parts = text.split('\n')
    return [parse_ints(p) for p in parts[1:] if p != '']
    
def parse_problem(text):
    parts = text.split('\n\n')
    seeds = parse_ints(parts[0].split(': ')[1])
    mappings = [parse_mapping(p) for p in parts[1:]]
    return seeds, mappings

def map_seed(seed, mappings):
    for m in mappings:
        for d, s, l in m:
            if seed >= s and seed < s + l:
                seed = d + seed - s
                break
    return seed

def map_range(range, mapping):
    ranges = set()
    ranges.add(range)
    res = []
    while len(ranges) > 0:
        r = ranges.pop()
        ranges.add(r)
        mapped = False
        for d, s, l in mapping:
            rs, rl = r
            if (s < rs + rl) and (s + l > rs):
                p1 = d + max(s, rs) - s
                p2 = d + min(s + l, rs + rl) - s
                res.append((p1, p2 - p1))
                mapped = True
                # clip the range:
                ranges.remove(r)
                if rs < s:
                    ranges.add((rs, s - rs))
                if rs + rl > s + l:
                    ranges.add((s + l, rs + rl - s - l))
                break
        if not mapped:
            break
    res += list(ranges)
    return res

def map_ranges(ranges, mappings):
    for mapping in mappings:
        res = []
        for r in ranges:
            res += map_range(r, mapping)
            ranges = res
    return ranges

seeds, mappings = parse_problem(text)

res1 = min(map_seed(s, mappings) for s in seeds)
print(res1)

ranges = [(seeds[i * 2], seeds[i * 2 + 1]) for i in range(int(len(seeds)/2))]
res2 = min(x for x, _ in map_ranges(ranges, mappings))
print(res2)

In [ ]:
# DAY06
import math
from operator import mul
from functools import reduce

f = open('input/06.txt', 'r')
lines = [line.strip() for line in f.readlines()]

def parse_ints(line):
    return [int(x.strip()) for x in line.split()]

def parse_input(lines):
    t = parse_ints(lines[0].split(':')[1])
    d = parse_ints(lines[1].split(':')[1])
    return [(t[i], d[i]) for i in range(len(t))]

def get_roots_dist(t, d):
    det = t * t - 4 * d
    if det < 0:
        return 0
    ds = math.sqrt(det)
    r1 = math.ceil(0.5 * (ds + t))
    r2 = math.floor(0.5 * (t - ds))
    return abs(r1 - r2 - 1)
    
races = parse_input(lines)
dists = [get_roots_dist(t, d) for t, d in races]
res1 = reduce(mul, dists, 1)
print(res1)

lines2 = parse_input([l.replace(' ', '') for l in lines])
res2 = get_roots_dist(races2[0][0], races2[0][1])
print(res2)

In [ ]:
# DAY07
from functools import cmp_to_key

f = open('input/07.txt', 'r')
lines = [line.strip() for line in f.readlines()]


def parse_card(line):
    p = line.split()
    return (p[0], int(p[1]))


def get_hist(card):
    res = {}
    for c in card:
        if c in res:
            res[c] += 1
        else:
            res[c] = 1
    return res


def get_hand_id_raw(card):
    h = get_hist(card)
    f = sorted(h.values(), reverse=True)
    if f[0] == 5:
        return 0
    elif f[0] == 4:
        return 1
    elif f[0] == 3 and f[1] == 2:
        return 2
    elif f[0] == 3:
        return 3
    elif f[0] == 2 and f[1] == 2:
        return 4
    elif f[0] == 2:
        return 5
    elif len(f) == 5:
        return 6
    else:
        return 7


def get_hand_id(card, use_joker=False):
    if not use_joker:
        return get_hand_id_raw(card)
    return min(get_hand_id_raw(card.replace('J', c)) for c in get_hist(card).keys())


def compare_cards(ca,  cb, ranks_map, use_joker=False):
    a, _ = ca
    b, _ = cb
    h1 = get_hand_id(a, use_joker)
    h2 = get_hand_id(b, use_joker)
    if h1 < h2:
        return -1
    elif h1 > h2:
        return 1
    for i in range(len(a)):
        r1 = ranks_map[a[i]]
        r2 = ranks_map[b[i]]
        if r1 < r2:
            return -1
        elif r1 > r2:
            return 1
    return 0


RANKS = "AKQJT98765432"
RANKS_MAP = {RANKS[i]: i for i in range(len(RANKS))}


def compare_cards1(ca, cb):
    return compare_cards(ca, cb, RANKS_MAP, False)


RANKS2 = "AKQT98765432J"
RANKS_MAP2 = {RANKS2[i]: i for i in range(len(RANKS2))}


def compare_cards2(ca, cb):
    return compare_cards(ca, cb, RANKS_MAP2, True)


def get_score(cards):
    return sum(cards[i][1] * (i + 1) for i in range(len(cards)))


cards = [parse_card(line) for line in lines]

cards = sorted(cards, key=cmp_to_key(compare_cards1), reverse=True)
res1 = get_score(cards)
print(res1)

cards = sorted(cards, key=cmp_to_key(compare_cards2), reverse=True)
res2 = get_score(cards)
print(res2)

In [ ]:
# DAY08
import math
import functools 

f = open('input/08.txt', 'r')
lines = [line.strip() for line in f.readlines()]

def parse_node(line):
    parts = line.split(' = ')
    return parts[0], parts[1][1:-1].split(', ')

def lcm(a, b):
    return abs(a*b) // math.gcd(a, b)

def get_nsteps(n='AAA', lastz=True):
    i = 0
    while True:
        s = dirs[i % len(dirs)]
        idx = 0 if s == 'L' else 1
        n = nodes[n][idx]
        i += 1
        if (not lastz and n == 'ZZZ') or (lastz and n[-1] == 'Z'):
            return i

dirs = lines[0]
nodes = [parse_node(line) for line in lines[2:]]
nodes = {n: lr for n, lr in nodes}

res1 = get_nsteps('AAA', False)
print(res1)

start_nodes = [x for x in nodes.keys() if x[-1] == 'A']
nsteps = [get_nsteps(n, True) for n in start_nodes]
res2 = functools.reduce(lambda a, b: lcm(a, b), nsteps)
print(res2)

In [ ]:
# DAY09

f = open('input/09.txt', 'r')


def parse_ints(line):
    return [int(x.strip()) for x in line.split()]


def get_derivatives(nums):
    n = len(nums)
    res = [0] * (n - 1)
    for i in range(n - 1):
        res[i] = nums[i + 1] - nums[i]
    return res
    

def extrapolate(nums):
    derivatives = [nums]
    n = nums
    while True:
        d = get_derivatives(n)
        if all(x == 0 for x in d):
            res1, res2 = 0, 0
            nd = len(derivatives)
            for i in range(nd):
                res1 = derivatives[nd - 1 - i][0] - res1
                res2 += derivatives[nd - 1 - i][-1]
            return res1, res2
        derivatives.append(d)
        n = d

nums = [parse_ints(line.strip()) for line in f.readlines()]

extr = [extrapolate(n) for n in nums]
res1 = sum(b for _, b in extr)
print(res1)

res2 = sum(a for a, _ in extr)
print(res2)

In [ ]:
# DAY10

f = open('input/10_test2.txt', 'r')
field = [line.strip() for line in f.readlines()]

DIRS = [(0, 1), (1, 0), (-1, 0), (0, -1)]
SYMS = {
    '|': [(0, -1), (0, 1)],
    '-': [(-1, 0), (1, 0)],
    'L': [(0, -1), (1, 0)],
    'J': [(0, -1), (-1, 0)],
    '7': [(0, 1), (-1, 0)],
    'F': [(0, 1), (1, 0)],
}

def find_start(field):
    w = len(field[0])
    h = len(field)
    for j in range(h):
        for i in range(w):
            if field[j][i] == 'S':
                return i, j


def find_next(field, start):
    w, h = len(field[0]), len(field)
    x, y = start
    for dx, dy in DIRS:
        px, py = x + dx, y + dy
        if px < 0 or px >= w or py < 0 or py >= h:
            continue
        c = field[py][px]
        if not c in SYMS:
            continue
        for kx, ky in SYMS[c]:
            nx, ny = px + kx, py + ky
            if (nx, ny) == (x, y):
                yield px, py

def get_sym(field, pos):
    x, y = pos
    c = field[y][x]
    if c != 'S':
        return c
    adj = sorted([(px - x, py - y) for px, py in find_next(field, pos)])
    for k, v in SYMS.items():
        if adj == sorted(v):
            return k

def traverse(field):
    w, h = len(field[0]), len(field)
    prev = find_start(field)
    x, y = list(find_next(field, prev))[0]
    yield prev
    while True:
        c = field[y][x]
        for dx, dy in SYMS[c]:
            px, py = x + dx, y + dy
            if (px, py) != prev:
                prev = x, y
                x, y = px, py
                break
        yield prev
        if field[y][x] == 'S' or (x, y) == prev:
            return

def get_inside(field, path):
    w, h = len(field[0]), len(field)
    res = 0
    spath = set(path)
    for j in range(h):
        inside = False
        i = 0
        while i < w:
            c = get_sym(field, (i, j))
            if (i, j) in spath:
                if c == '|':
                    inside = not inside
                elif c == 'F':
                    inside = not inside
                    while get_sym(field, (i, j)) not in ['J', '7']:
                        i += 1
                    if get_sym(field, (i, j)) == '7':
                        inside = not inside
                elif c == 'L':
                    inside = not inside
                    while get_sym(field, (i, j)) not in ['J', '7']:
                        i += 1
                    if get_sym(field, (i, j)) == 'J':
                        inside = not inside
            else:
                if inside:
                    yield i, j
            i += 1

path = [p for p in traverse(field)]
res1 = int(len(path) / 2)
print(res1)
    
inside = list(get_inside(field, path))
res2 = len(inside)
print(res2)

def draw_field(field, path, inside):
    SUBST = {
        '|': '│',
        '-': '─',
        'L': '└',
        'J': '┘',
        '7': '┐',
        'F': '┌',
    }
    spath = set(path)
    sinside = set(inside)
    for j in range(len(field)):
        line = field[j]
        chars = [c for c in line]
        for i in range(len(chars)):
            c = chars[i]
            if c == 'S':
                continue
            if (i, j) not in spath:
                chars[i] = '█' if (i, j) in sinside else ' '
            else:
                chars[i] = SUBST[c]
        print(''.join(chars))

draw_field(field, path, inside)

In [ ]:
# DAY11

f = open('input/11.txt', 'r')
lines = [line.strip() for line in f.readlines()]


def parse_field(lines):
    h = len(lines)
    w = len(lines[0])
    res = set()
    for j in range(h):
        for i in range(w):
            if lines[j][i] == '#':
                res.add((i, j))
    return res


def manhattan_dist(a, b):
    ax, ay = a
    bx, by = b
    return abs(ax - bx) + abs(ay - by)


def expand_field(field, factor=2):
    sf = sorted(field)
    res = set()
    lastx = sf[0][0]
    delta = 0
    for x, y in sf:
        if x - lastx > 1:
            delta += (factor - 1) * (x - lastx - 1)
        res.add((x + delta, y))
        lastx = x
    return res

def swap_coords(field):
    res = set()
    for x, y in field:
        res.add((y, x))
    return res

def get_dists(field):
    coords = list(field)
    nc = len(coords)
    res = 0
    for i in range(nc):
        for j in range(i + 1, nc):
            res += manhattan_dist(coords[i], coords[j])
    return res

def solve(field, factor=2):
    field = expand_field(field, factor)
    field = swap_coords(expand_field(swap_coords(field), factor))
    return get_dists(field)

field = parse_field(lines)
res1 = solve(field)
print(res1)

res2 = solve(field, 1000000)
print(res2)

In [ ]:
%%time
# DAY12

f = open('input/12.txt', 'r')
lines = [line.strip() for line in f.readlines()]

def parse_line(line):
    parts = line.split()
    return list(parts[0]), [int(x.strip()) for x in parts[1].strip().split(",")]

def get_num_subst(syms, ranges, mem, syms_offs=0, range_offs=0, cont=False):
    nr = len(ranges)
    if syms_offs == len(syms):
        if range_offs == nr or (range_offs == nr - 1 and 
                                ranges[range_offs] == 0):
            return 1
        else:
            return 0
    c = syms[syms_offs]
    key = (syms_offs, range_offs, -1 if not cont or range_offs >= nr else ranges[range_offs], c)
    if key in mem:
        return mem[key]

    res = 0
    if c == '#':
        if range_offs == nr or ranges[range_offs] == 0:
            return 0
        ranges[range_offs] -= 1
        res = get_num_subst(syms, ranges, mem, syms_offs + 1, range_offs, True)
        ranges[range_offs] += 1
    elif c == '.':
        if range_offs < nr and ranges[range_offs] != 0 and cont:
            return 0
        if range_offs < nr and ranges[range_offs] == 0:
            res = get_num_subst(syms, ranges, mem, syms_offs + 1, range_offs + 1, False)
        else:
            res = get_num_subst(syms, ranges, mem, syms_offs + 1, range_offs, False)
    elif c == '?':
        syms[syms_offs] = '#'
        n1 = get_num_subst(syms, ranges, mem, syms_offs, range_offs, cont)
        syms[syms_offs] = '.'
        n2 = get_num_subst(syms, ranges, mem, syms_offs, range_offs, cont)
        syms[syms_offs] = '?'
        res = n1 + n2
        
    mem[key] = res
    return res

def expand(e, n=5):
    return list("?".join(["".join(e[0])] * n)), e[1] * n


entries = [parse_line(line) for line in lines]
res1 = sum(get_num_subst(s, r, {}) for s, r in entries)
print(f"Answer 1: {res1}")

entries_expanded = [expand(e, 5) for e in entries]
res2 = sum(get_num_subst(s, r, {}) for s, r in entries_expanded)
print(f"Answer 2: {res2}")

In [ ]:
%%time
# DAY13
data = open('input/13.txt', 'r').read()


def parse_field(data):
    lines = data.strip().split('\n')
    return [list(line) for line in lines]


def transpose(field):
    w = len(field[0])
    h = len(field)
    res = [[' '] * h for _ in range(w)]
    for j in range(h):
        for i in range(w):
            res[i][j] = field[j][i]
    return res


def find_mirrors(field):
    w = len(field[0])
    h = len(field)
    hashes = [0] * h
    hreg = {}
    for j in range(h):
        ch = hash("".join(field[j]))
        if ch in hreg and hreg[ch] != field[j]:
            raise ValueError('Hash collision!!!')
        hashes[j] = ch
        hreg[ch] = field[j]
    for j in range(h - 1):
        n = 0
        while j - n >= 0 and j + n < h - 1 and hashes[j - n] == hashes[j + 1 + n]:
            n += 1
        if n == min(j + 1, h - j - 1):
            yield j + 1


def eval_reflections(fields):
    res = 0
    for f in fields:
        s1 = 100 * sum(s for s in find_mirrors(f))
        s2 = sum(s for s in find_mirrors(transpose(f)))
        res += s1 + s2
    return res


def flip_cell(field, x, y):
    c = field[y][x]
    field[y][x] = '#' if c == '.' else '.'


def fix_smudge(field):
    w = len(field[0])
    h = len(field)
    tfield = transpose(field)
    hlines = set(find_mirrors(field))
    vlines = set(find_mirrors(tfield))
    for j in range(h):
        for i in range(w):
            flip_cell(field, i, j)
            flip_cell(tfield, j, i)
            hlines1 = set(find_mirrors(field))
            vlines1 = set(find_mirrors(tfield))
            if len(hlines1) + len(vlines1) > 0 and (hlines != hlines1 or vlines != vlines1):
                yield hlines1.difference(hlines), vlines1.difference(vlines)
            flip_cell(field, i, j)
            flip_cell(tfield, j, i)


fields = [parse_field(p) for p in data.split('\n\n')]

res1 = eval_reflections(fields)
print(res1)

res2 = 0
for f in fields:
    hlines, vlines = list(fix_smudge(f))[0]
    s1 = 100 * sum(list(hlines))
    s2 = sum(list(vlines))
    res2 += s1 + s2
print(res2)


In [ ]:
# DAY14
f = open('input/14.txt', 'r')
field = [list(line.strip()) for line in f.readlines()]


def tilt_v(field, dir):
    w, h = len(field[0]), len(field)
    offs = [0] * w
    for j in range(h) if dir == -1 else range(h - 1, -1, -1):
        for i in range(w):
            c = field[j][i]
            if c == '.':
                offs[i] += 1
            elif c == '#':
                offs[i] = 0
            elif c == 'O':
                field[j][i] = '.'
                field[j + dir * offs[i]][i] = 'O'


def tilt_h(field, dir):
    w, h = len(field[0]), len(field)
    offs = [0] * h
    for j in range(h):
        for i in range(w) if dir == -1 else range(w - 1, -1, -1):
            c = field[j][i]
            if c == '.':
                offs[j] += 1
            elif c == '#':
                offs[j] = 0
            elif c == 'O':
                field[j][i] = '.'
                field[j][i + dir * offs[j]] = 'O'


def cycle(field):
    tilt_v(field, -1)
    tilt_h(field, -1)
    tilt_v(field, 1)
    tilt_h(field, 1)


def cycle_times(field, iters):
    history = {}
    rem = 0
    for i in range(iters):
        key = hash("".join("".join(p) for p in field))
        if key in history:
            # found loop
            start = history[key]
            rem = (iters - start) % (i - start)
            break
        cycle(field)
        history[key] = i
    
    for i in range(rem):
        cycle(field)


def compute_load(field):
    w, h = len(field[0]), len(field)
    res = 0
    for j in range(h):
        for i in range(w):
            c = field[j][i]
            if c == 'O':
                res += h - j
    return res
    

field1 = [row[:] for row in field]
tilt_v(field1, -1)
res1 = compute_load(field1)
print(res1)

cycle_times(field, 1000000000)
res2 = compute_load(field)
print(res2)

In [ ]:
# DAY15
import re
data = open('input/15.txt', 'r').read()


def compute_hash(s):
    res = 0
    for c in s:
        res = (res + ord(c)) * 17 % 256
    return res

def parse_cmd(part):
    p = re.split(r'-|=', part)
    if p[1] == '':
        return (p[0], None)
    else:
        return (p[0], int(p[1]))


def eval_cmds(cmds):
    boxes = [([], {}) for i in range(256)]
    for label, n in cmds:
        id = compute_hash(label)
        lst, reg = boxes[id]
        if label not in reg:
            reg[label] = len(lst)
            lst.append((label, n))
        lst[reg[label]] = (label, n)
        if n is None:
            del reg[label]
    return boxes
    

def compute_score(boxes):
    res = 0
    for i in range(len(boxes)):
        lst, reg = boxes[i]
        offs = 0
        for j in range(len(lst)):
            _, n = lst[j]
            if n is None:
                offs += 1
            else:
                res += (i + 1) * (j + 1 - offs) * n
    return res


parts = data.strip().split(',')
res1 = sum(compute_hash(s) for s in parts)
print(res1)

cmds = [parse_cmd(p) for p in parts]
boxes = eval_cmds(cmds)
res2 = compute_score(boxes)
print(res2)

In [ ]:
%%time
# DAY16
f = open('input/16.txt', 'r')
field = [line.strip() for line in f.readlines()]

DIRS = [(1, 0), (0, 1), (-1, 0), (0, -1)]


def trace_ray(field, pos, dir, visited):
    w, h = len(field[0]), len(field)
    x, y = pos
    while (x, y, dir) not in visited:
        c = field[y][x]
        visited.add((x, y, dir))
        if c == '-':
            if dir in [1, 3]:
                trace_ray(field, (x, y), 0, visited)
                dir = 2
        elif c == '|':
            if dir in [0, 2]:
                trace_ray(field, (x, y), 1, visited)
                dir = 3
        elif c == '\\':
            dir = [1, 0, 3, 2][dir]
        elif c == '/':
            dir = [3, 2, 1, 0][dir]
        dx, dy = DIRS[dir]
        x, y = x + dx, y + dy
        if x < 0 or y < 0 or x >= w or y >= h:
            break
            

def get_num_visited(field, pos, dir):
    visited = set()
    trace_ray(field, pos, dir, visited)
    return len(set((x, y) for x, y, _ in visited))


res1 = get_num_visited(field, (0, 0), 0)
print(res1)

res2 = 0
w, h = len(field[0]), len(field)
for i in range(w):
    res2 = max(res2, get_num_visited(field, (i, 0), 1))
    res2 = max(res2, get_num_visited(field, (i, h - 1), 3))
for i in range(h):
    res2 = max(res2, get_num_visited(field, (0, i), 0))
    res2 = max(res2, get_num_visited(field, (w - 1, i), 2))
print(res2)

In [ ]:
%%time
# DAY17
import heapq

f = open('input/17.txt', 'r')
field = [line.strip() for line in f.readlines()]


def find_min_path(field, src, dst, min_steps, max_steps):
    w, h = len(field[0]), len(field)
    open = [(0, (src, 0, (0, 0)), src)]
    visited = {}
    while len(open) > 0:
        weight, node, prev = heapq.heappop(open)
        pos, steps, dir = node
        if node in visited and visited[node][0] != weight:
            continue
        x, y = pos
        px, py = prev
        next_dirs = []
        sx, sy = dir
        if steps >= min_steps or dir == (0, 0):
            if sy == 0:
                next_dirs += [(0, 1, 0), (0, -1, 0)]
            if sx == 0:
                next_dirs += [(-1, 0, 0), (1, 0, 0)]
        if steps < max_steps and dir != (0, 0):
            next_dirs += [(sx, sy, steps)]
            
        for dx, dy, csteps in next_dirs:
            cx, cy = x + dx, y + dy
            cpos = (cx, cy)
            if cx < 0 or cy < 0 or cx >= w or cy >= h:
                continue
            cw = weight + int(field[cy][cx])
            cnode = cpos, csteps + 1, (dx, dy)
            if cnode in visited and visited[cnode][0] <= cw:
                continue
            if cpos == dst and csteps + 1 < min_steps:
                continue
            heapq.heappush(open, (cw, cnode, pos))
            visited[cnode] = (cw, pos, steps)
    res = min(visited[key] for key in visited if key[0] == dst)
    return res[0]


w, h = len(field[0]), len(field)
res1 = find_min_path(field, (0, 0), (w - 1, h - 1), 0, 3)
print(res1)

res2 = find_min_path(field, (0, 0), (w - 1, h - 1), 4, 10)
print(res2)

In [2]:
# DAY18

f = open('input/18.txt', 'r')
lines = [line.strip() for line in f.readlines()]

DIRS = {
    'U': ((0, -1), (0, 0)),
    'R': ((1, 0), (0, 0)), 
    'D': ((0, 1), (1, 0)),
    'L':  ((-1, 0), (0, 1)),
}

def parse_line(line):
    parts = line.translate({ord(i): None for i in '#()'}).split()
    return tuple(parts)


def trace(cmds):
    res = [(0, 0)]
    x, y = 0, 0
    for dir, n, _ in cmds:
        (dx, dy), (lx, ly) = DIRS[dir]
        dist = int(n)
        x, y = x + dx * dist, y + dy * dist
        ex, ey = x + lx, y + ly
        # expand previous point to cell boundaries as well
        px, py = res[-1]
        if px == x:
            px = ex
        if py == y:
            py = ey
        res[-1] = px, py
        res.append((ex, ey))
    return res


def translate_cmds(cmds):
    res = []
    for _, _, clr in cmds:
        dir = "RDLU"[int(clr[-1])]
        dist = int(clr[:-1], 16)
        res.append((dir, dist, clr))
    return res
    

def scan_points(points):
    hsegs = []
    np = len(points)
    for i in range(0, np - 1):
        p1 = points[i]
        p2 = points[(i + 1) % np]
        if p1[1] == p2[1]:
            hsegs.append((p1, p2))
            
    pts = []
    for i in range(len(hsegs)):
        p1, p2 = hsegs[i]
        px1, px2 = p1[0], p2[0]
        if px1 > px2:
            px1, px2 = px2, px1
        pts.append((px1, i + 1))
        pts.append((px2, -(i + 1)))
    pts = sorted(pts)

    res = 0
    prevx = pts[0][0]
    ycoords = set()
    for x, id in pts:
        y = hsegs[abs(id) - 1][0][1]
        if x != prevx:
            i = 0
            yy = sorted(list(ycoords))
            while i < len(yy):
                res += (x - prevx) * (yy[i + 1] - yy[i])
                i += 2
        if id > 0:
            ycoords.add(y)
        else:
            ycoords.remove(y)
        prevx = x
    return res


cmds = [parse_line(line) for line in lines]
points = trace(cmds)
res1 = scan_points(points)
print(res1)

points = trace(translate_cmds(cmds))
res2 = scan_points(points)
print(res2)

61865
40343619199142


In [22]:
# DAY19
import re
from operator import mul
from functools import reduce

data = open('input/19.txt', 'r').read()

def parse_rule(line):
    p = line.split('{')
    name = p[0]
    conds = []
    for cp in p[1][:-1].split(','):
        lr = cp.split(':')
        if len(lr) == 1:
            conds.append((None, lr[0]))
        else:
            r, sign, n = re.split(r"(>|<)", lr[0])
            conds.append(((r, sign, int(n)), lr[1]))
    return name, conds
    

def parse_part(line):
    parts = []
    for p in line[1:-1].split(','):
        _, val = p.split('=')
        parts.append(int(val))
    return parts
    

def parse_data(data):
    parts = data.split('\n\n')
    p0 = dict(parse_rule(p) for p in parts[0].split('\n') if p != '')
    p1 = [parse_part(p) for p in parts[1].split('\n') if p != '']
    return p0, p1


PART_MAPPING = {
    'x': 0,
    'm': 1,
    'a': 2,
    's': 3,
}

def eval_part(part, rules):
    rule = 'in'
    while True:
        for cond in rules[rule]:
            cmp, next = cond
            if cmp != None:
                r, sign, n = cmp
                val = part[PART_MAPPING[r]]
                if sign == '<' and val >= n:
                    continue
                elif sign == '>' and val <= n:
                    continue
            if next == 'A':
                return True
            elif next == 'R':
                return False
            else:
                rule = next
                break

def count_num_accepted_rec(rules, ranges, rule):
    if rule == 'A':
        return reduce(mul, [max(0, maxv - minv + 1) for minv, maxv in ranges], 1)
    elif rule == 'R':
        return 0
    cranges = [[minv, maxv] for minv, maxv in ranges]
    res = 0
    for cond in rules[rule]:
        cmp, next = cond
        if cmp != None:
            r, sign, n = cmp
            idx = PART_MAPPING[r]
            ranges1 = [[minv, maxv] for minv, maxv in cranges]
            if sign == '<':
                ranges1[idx][1] = min(ranges1[idx][1], n - 1)
                cranges[idx][0] = max(n, cranges[idx][0])
                res += count_num_accepted_rec(rules, ranges1, next)
            elif sign == '>':
                ranges1[idx][0] = max(ranges1[idx][0], n + 1)
                cranges[idx][1] = min(n, cranges[idx][1])
                res += count_num_accepted_rec(rules, ranges1, next)
        else:
            res += count_num_accepted_rec(rules, cranges, next)
    return res

def count_num_accepted(rules, minv, maxv):
    ranges = [[minv, maxv]] * 4
    return count_num_accepted_rec(rules, ranges, 'in')


rules, parts = parse_data(data)
res1 = sum(sum(p) for p in parts if eval_part(p, rules))
print(res1)

res2 = count_num_accepted(rules, 1, 4000)
print(res2)

391132
128163929109524


In [37]:
%%time
# DAY20
from collections import deque
import math
import functools 

f = open('input/20.txt', 'r')
lines = [line.strip() for line in f.readlines()]

TYPE_NONE = 0
TYPE_FLIPFLOP = 1
TYPE_CONJ = 2

SIG_LOW = 0
SIG_HIGH = 1


def parse_line(line):
    parts = line.split('->')
    name = parts[0].strip()
    mtype = TYPE_NONE
    if name[0] == '%':
        mtype = TYPE_FLIPFLOP
    elif name[0] == '&':
        mtype = TYPE_CONJ
    if mtype != TYPE_NONE:
        name = name[1:]
    outputs = []
    if len(parts) > 1:
        outputs = [x.strip() for x in parts[1].split(', ') if x != '']
    return name, (mtype, outputs)


def get_inputs(mods):
    res = {}
    for mname, (_, outputs) in mods.items():
        for mout in outputs:
            if mout not in res:
                res[mout] = {}
            res[mout][mname] = SIG_LOW
    return res
    

def eval(mods, times=1, wait_for_sig=None, start_mod_name='broadcaster', start_sig_type=SIG_LOW):
    minputs = get_inputs(mods)
    mstate = {mname: [False, minputs[mname]] for mname in minputs.keys()}
    sig_counts = {SIG_LOW: 0, SIG_HIGH: 0}

    presses = 0
    while True:
        #print("--- iter", i)
        mq = deque()
        mq.append((start_mod_name, start_sig_type))
        while len(mq) > 0:
            mname, stype = mq.popleft()
            sig_counts[stype] += 1
            if mname == wait_for_sig and stype == SIG_LOW:
                return sig_counts, presses + 1
            if mname not in mods:
                continue
            mtype, outputs = mods[mname]
            out_stype = stype
            if mtype == TYPE_FLIPFLOP:
                if stype == SIG_LOW:
                    state = mstate[mname][0]
                    if state:
                        out_stype = SIG_LOW
                        mstate[mname][0] = False
                    else:
                        out_stype = SIG_HIGH
                        mstate[mname][0] = True
                else:
                    continue
            elif mtype == TYPE_CONJ:
                inputs = mstate[mname][1]
                if all(s == SIG_HIGH for s in inputs.values()):
                    out_stype = SIG_LOW
                else:
                    out_stype = SIG_HIGH
    
            for mout in outputs:
                mstate[mout][1][mname] = out_stype
                mq.append((mout, out_stype))
        presses += 1
        if times > 0 and presses == times:
            break
    return sig_counts, presses


def lcm(a, b):
    return abs(a*b) // math.gcd(a, b)


def lcm_arr(arr):
    return functools.reduce(lambda a, b: lcm(a, b), arr)


mods = {name: conf for name, conf in map(parse_line, lines)}
counts, _ = eval(mods, 10000)
cv = list(counts.values())
res1 = counts[0] * counts[1]
print(res1)

minputs = get_inputs(mods)
in1 = list(minputs['rx'].keys())[0]
in2 = minputs[in1].keys()
steps = []
for mname in in2:
    _, s = eval(mods, -1, mname)
    steps.append(s)
res2 = lcm_arr(steps)
print(res2)
    

89194858006
228134431501037
CPU times: total: 984 ms
Wall time: 961 ms


The logic of part 2 above is inferred by looking at the actual graph from input and its signal propagation:
![](media/20.png)

In [3]:
%%time
# DAY21

f = open('input/21.txt', 'r')
field = [list(line.strip()) for line in f.readlines()]

DIRS = [
    (1, 0),
    (0, -1),
    (-1, 0),
    (0, 1),
]

def find_start(field):
    w, h = len(field[0]), len(field)
    for j in range(h):
        for i in range(w):
            if field[j][i] == 'S':
                return i, j


def step(field, ppos):
    w, h = len(field[0]), len(field)
    res = set()
    for x, y in ppos:
        for dx, dy in DIRS:
            cx, cy = x + dx, y + dy
            if field[cy % h][cx % w] != '#':
                res.add((cx, cy))
    return res
    

def count_inside(ppos, xmin, xmax, ymin, ymax):
    return sum(x >= xmin and x <= xmax and y >= ymin and y <= ymax
               for x, y in ppos)
    

def step_times(field, n):
    sx, sy = find_start(field)
    ppos = set()
    ppos.add((sx, sy))
    for i in range(n):
        ppos = step(field, ppos)
    return ppos

def eval_pcount(ppos, side, nsteps):
    def count_in_square(x, y):
        return count_inside(ppos, x * side, (x + 1) * side - 1, y * side, (y + 1) * side - 1)

    # full squares odd/even
    nc1 = count_in_square(0, 0)
    nc2 = count_in_square(1, 0)

    # tip squares
    nt = count_in_square(0, -2)
    nl = count_in_square(-2, 0)
    nb = count_in_square(0, 2)
    nr = count_in_square(2, 0)

    # edge squares odd
    nlt1 = count_in_square(-1, -1)
    nrt1 = count_in_square(1, -1)
    nrb1 = count_in_square(1, 1)
    nlb1 = count_in_square(-1, 1)

    # ege squares even
    nlt2 = count_in_square(-2, -1)
    nrt2 = count_in_square(2, -1)
    nrb2 = count_in_square(2, 1)
    nlb2 = count_in_square(-2, 1)
    
    n = (nsteps - s0) // side
    
    res = nt + nl + nb + nr
    res += nc1 * ((n - 1) ** 2) + nc2 * (n ** 2) 
    res += (nlt1 + nrt1 + nrb1 + nlb1) * (n - 1) + (nlt2 + nrt2 + nrb2 + nlb2) * n 
    return res

ppos = step_times(field, 64)
res1 = len(ppos)
print(res1)

side = len(field)
s0 = side // 2
ppos = step_times(field, s0 + side * 2)

NSTEPS = 26501365
res2 = eval_pcount(ppos, side, NSTEPS)
print(res2)

3770
628206330073385
CPU times: total: 14.9 s
Wall time: 14.9 s
